In [5]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('mode.chained_assignment', None)

In [6]:
startime=pd.Timestamp.now() 
df_refdata = pd.read_parquet('./raw/refdata.parquet')
df_marketdata = pd.read_parquet('./raw/marketdata.parquet')
df_executions = pd.read_parquet('./raw/exectuions.parquet')

In [7]:
df_executions.describe(include = 'all')

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity
count,4203,4203,4203,4203,4203.000000,4203.00000,4203,4203.000000
unique,66,3,6,3846,NaN,NaN,2,NaN
top,DE000BASF111,EUR,XETA,2022-09-02 15:31:39.000,NaN,NaN,CONTINUOUS_TRADING,NaN
freq,483,3349,2904,14,NaN,NaN,4103,NaN
mean,NaN,NaN,NaN,NaN,431.903731,2101.00000,NaN,7.749465
std,NaN,NaN,NaN,NaN,2252.383559,1213.44592,NaN,352.150518
min,NaN,NaN,NaN,NaN,1.380000,0.00000,NaN,-12780.000000
25%,NaN,NaN,NaN,NaN,41.980000,1050.50000,NaN,-26.000000
50%,NaN,NaN,NaN,NaN,52.070000,2101.00000,NaN,11.000000
75%,NaN,NaN,NaN,NaN,82.100000,3151.50000,NaN,39.000000


In [8]:
df_refdata.describe(include = 'all')

,ISIN,id,Currency,primary_ticker,primary_mic
count,262,2.620000e+02,262,262,262
unique,260,NaN,7,261,17
top,DK0061539921,NaN,EUR,LIN,XLON
freq,2,NaN,114,2,50
mean,NaN,2.439424e+07,NaN,NaN,NaN
std,NaN,8.786413e+07,NaN,NaN,NaN
min,NaN,1.212100e+05,NaN,NaN,NaN
25%,NaN,3.239355e+05,NaN,NaN,NaN
50%,NaN,3.272450e+05,NaN,NaN,NaN
75%,NaN,3.504150e+05,NaN,NaN,NaN


In [9]:
df_marketdata.describe(include='all')

C:\Users\patel\AppData\Local\Temp\ipykernel_14536\4058298076.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_marketdata.describe(include='all')


,event_timestamp,best_bid_size,best_bid_price,best_ask_price,best_ask_size,market_state,primary_mic,listing_id
count,1916902,1.916902e+06,1.916895e+06,1.916895e+06,1.916902e+06,1916902,1916902,1.916902e+06
unique,1916868,NaN,NaN,NaN,NaN,8,3,NaN
top,2022-09-02 15:20:00.000017,NaN,NaN,NaN,NaN,CONTINUOUS_TRADING,XSWX,NaN
freq,7,NaN,NaN,NaN,NaN,1916113,1220442,NaN
first,2022-09-02 01:00:57.205489803,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-09-02 19:14:40.907065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,5.692193e+02,1.400823e+02,1.401319e+02,5.254349e+02,NaN,NaN,3.203729e+06
std,NaN,7.883578e+02,1.408631e+02,1.409117e+02,6.356345e+02,NaN,NaN,3.014639e+07
min,NaN,0.000000e+00,1.958000e+01,1.978000e+01,0.000000e+00,NaN,NaN,3.234360e+05
25%,NaN,1.530000e+02,4.771000e+01,4.773000e+01,1.490000e+02,NaN,NaN,3.234780e+05


In [10]:
executions_cnt=df_executions.shape[0]

# 1. Start:
  a. Count the number of executions within the executions.parquet file, determine the unique number of [‘Venue’]s and the date of executions. Log output this information.

In [11]:
executions_cnt

4203

In [12]:
df_executions['Trade_Dt']=pd.to_datetime(df_executions['TradeTime']).dt.date

In [13]:
df_executions[['Venue','Trade_Dt']].drop_duplicates()

,Venue,Trade_Dt
0,XETA,2022-09-02
76,XCSE,2022-09-02
82,XBRU,2022-09-02
280,XSWX,2022-09-02
1279,XETS,2022-09-02
1282,XETB,2022-09-02


determine the unique number of [‘Venue’]s and the date of executions. Log output this information

In [14]:
Unique_Cnt=df_executions[['Venue','Trade_Dt']].drop_duplicates().shape[0]
Unique_Cnt

6

# 2. Data Cleaning:
    a. Filter executions.paraquet for only CONTINUOUS_TRADING trades.

In [15]:
df_executions_filtered=df_executions[(df_executions['Phase']=='CONTINUOUS_TRADING')]

# 2. Data Cleaning:
    b.Log output the # of executions.

In [16]:
df_executions_filtered.shape[0]

4103

#     3. Data Transformation:
    a. Add column [‘side’], if quantity is negative, side = 2, if quantity is positive side = 1.

In [17]:
df_executions_filtered['Side']=np.where(df_executions_filtered['Quantity']<0, 2,1 )

    b. Complement the data with refdata.parquet 
       i. Add the primary ticker [‘primary_ticker’]
       ii. Add the primary mic [‘primary_mic’]

In [18]:
df_executions_filtered=pd.merge(df_executions_filtered,df_refdata,'left',left_on=['ISIN', 'Currency'], right_on=['ISIN', 'Currency'])

In [19]:
df_executions_filtered['Venue'].unique()

array(['XBRU', 'XETA', 'XSWX', 'XCSE', 'XETS', 'XETB'], dtype=object)

In [20]:
df_executions_filtered

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,Trade_Dt,Side,id,primary_ticker,primary_mic
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.80,100,CONTINUOUS_TRADING,11,2022-09-02,1,328336,AED,XBRU
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.90,101,CONTINUOUS_TRADING,16,2022-09-02,1,328336,AED,XBRU
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.85,102,CONTINUOUS_TRADING,22,2022-09-02,1,328336,AED,XBRU
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.95,103,CONTINUOUS_TRADING,17,2022-09-02,1,328336,AED,XBRU
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.05,104,CONTINUOUS_TRADING,23,2022-09-02,1,328336,AED,XBRU
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.66,4198,CONTINUOUS_TRADING,37,2022-09-02,1,326169,DHL,XETR
4099,DE0005552004,EUR,XETA,2022-09-02 15:29:58.824,36.65,4199,CONTINUOUS_TRADING,77,2022-09-02,1,326169,DHL,XETR
4100,DE0005552004,EUR,XETA,2022-09-02 15:35:26.504,36.60,4200,CONTINUOUS_TRADING,1279,2022-09-02,1,326169,DHL,XETR
4101,DE000A0LD6E6,EUR,XETA,2022-09-02 15:35:25.884,52.55,4201,CONTINUOUS_TRADING,-35,2022-09-02,2,327538,GXI,XETR


In [21]:
#convert the TradeTime to timestamp from str
df_executions_filtered['TradeTime'] = pd.to_datetime(df_executions_filtered['TradeTime'])

In [22]:
#convert event_timestamp to str and then make the nanosecond length in sync with execution table to correctly join the data, and then convert back to timestamp
df_marketdata['event_timestamp']=df_marketdata['event_timestamp'].values.astype(str)
df_marketdata['event_timestamp'] = df_marketdata['event_timestamp'].str[:23]
df_marketdata['event_timestamp'] = df_marketdata['event_timestamp'].str.replace('T',' ')
df_marketdata['event_timestamp'] = pd.to_datetime(df_marketdata['event_timestamp'])
df_marketdata

,event_timestamp,best_bid_size,best_bid_price,best_ask_price,best_ask_size,market_state,primary_mic,listing_id
0,2022-09-02 01:00:57.205,6740,23.00,24.75,91,CLOSED,XBRU,378534881
1,2022-09-02 01:00:57.210,26,31.80,32.50,65,CLOSED,XBRU,328259
2,2022-09-02 01:01:03.797,52,47.88,48.00,31,CLOSED,XBRU,324162
3,2022-09-02 01:01:03.802,281,30.65,33.33,300,CLOSED,XBRU,324168
4,2022-09-02 01:01:03.803,52,46.68,47.50,85,CLOSED,XBRU,324078
...,...,...,...,...,...,...,...,...
1916897,2022-09-02 16:22:29.648,50,315.00,319.00,190,POST_TRADE,XSWX,323508
1916898,2022-09-02 17:50:50.981,10,318.75,319.00,190,POST_TRADE,XSWX,323508
1916899,2022-09-02 17:56:45.473,50,315.00,319.00,190,POST_TRADE,XSWX,323508
1916900,2022-09-02 19:10:32.724,50,315.00,319.00,320,POST_TRADE,XSWX,323508


In [23]:
# # to test the data from df_marketdata_Test
# df_executions_filtered['TradeTime']=df_executions_filtered['TradeTime'].values.astype(str)
# df_executions_filtered[df_executions_filtered['TradeTime'].str.contains('2022-09-02T07:39:35')]

In [24]:
# # to test the data from df_marketdata_Test
# df_marketdata['event_timestamp']=df_marketdata['event_timestamp'].values.astype(str)
# df_marketdata[df_marketdata['event_timestamp'].str.contains('2022-09-02T07:39:40')].to_excel('check.xlsx')

In [25]:
# Checkpoint Dataframe before running the calculations
execution_df=df_executions_filtered.copy()
marketdata_df=df_marketdata.copy()
execution_df.reset_index(inplace=True, drop=True)
execution_df

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,Trade_Dt,Side,id,primary_ticker,primary_mic
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.80,100,CONTINUOUS_TRADING,11,2022-09-02,1,328336,AED,XBRU
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.90,101,CONTINUOUS_TRADING,16,2022-09-02,1,328336,AED,XBRU
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.85,102,CONTINUOUS_TRADING,22,2022-09-02,1,328336,AED,XBRU
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.95,103,CONTINUOUS_TRADING,17,2022-09-02,1,328336,AED,XBRU
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.05,104,CONTINUOUS_TRADING,23,2022-09-02,1,328336,AED,XBRU
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.66,4198,CONTINUOUS_TRADING,37,2022-09-02,1,326169,DHL,XETR
4099,DE0005552004,EUR,XETA,2022-09-02 15:29:58.824,36.65,4199,CONTINUOUS_TRADING,77,2022-09-02,1,326169,DHL,XETR
4100,DE0005552004,EUR,XETA,2022-09-02 15:35:26.504,36.60,4200,CONTINUOUS_TRADING,1279,2022-09-02,1,326169,DHL,XETR
4101,DE000A0LD6E6,EUR,XETA,2022-09-02 15:35:25.884,52.55,4201,CONTINUOUS_TRADING,-35,2022-09-02,2,327538,GXI,XETR


#     4. Calculations:
        a. Best bid price and best ask (bbo) – bbo data in marketdata.parquet

In [26]:
# Calculation for ‘best_bid’,’best_ask’
def find_bbo(timestamp,id):
   
    bbo_data = marketdata_df[(marketdata_df['event_timestamp'] == timestamp) & (marketdata_df['primary_mic'] == id)]
      
    if not bbo_data.empty:
        return bbo_data.iloc[0]['best_bid_price'], bbo_data.iloc[0]['best_ask_price']
    else:
        return None, None

# Find BBO prices for each execution
bbo_prices = []
for index, row in execution_df.iterrows():
    bbo = find_bbo(row['TradeTime'],row['primary_mic'])
    bbo_prices.append({'best_bid': bbo[0], 'best_ask': bbo[1]})

# Add BBO prices to the execution DataFrame
bbo_df = pd.DataFrame(bbo_prices)
execution_df = pd.concat([execution_df, bbo_df], axis=1)


In [27]:
# Calculation for ‘best_bid_min_1s’,’best_ask_min_1s' , here we also get rid of the nanoseconds from the timestamps as in many cases with nano seconds there were no values for -1 or +1 secs data.

marketdata_df['event_timestamp']=marketdata_df['event_timestamp'].values.astype('datetime64[s]')
execution_df['TradeTime']=execution_df['TradeTime'].values.astype('datetime64[s]')

def find_bbo_min_1s(timestamp,id):
    bbo_data = marketdata_df[(marketdata_df['event_timestamp'] == timestamp - pd.Timedelta(seconds=1)) & (marketdata_df['primary_mic'] == id)]
    
    if not bbo_data.empty:
        return bbo_data.iloc[0]['best_bid_price'], bbo_data.iloc[0]['best_ask_price']
    else:
        return None, None

# Find BBO prices for each execution
bbo_prices = []
for index, row in execution_df.iterrows():
    bbo = find_bbo_min_1s(row['TradeTime'],row['primary_mic'])
    bbo_prices.append({'best_bid_min_1s': bbo[0], 'best_ask_min_1s': bbo[1]})

# Add BBO prices to the execution DataFrame
bbo_df = pd.DataFrame(bbo_prices)
execution_df = pd.concat([execution_df, bbo_df], axis=1)


In [28]:
# Calculation for ‘best_bid_1s’,’best_ask_1s'
def find_bbo_1s(timestamp,id):
    bbo_data = marketdata_df[(marketdata_df['event_timestamp'] == timestamp + pd.Timedelta(seconds=1)) & (marketdata_df['primary_mic'] == id)]
    
    if not bbo_data.empty:
        return bbo_data.iloc[0]['best_bid_price'], bbo_data.iloc[0]['best_ask_price']
    else:
        return None, None

# Find BBO prices for each execution
bbo_prices = []
for index, row in execution_df.iterrows():
    bbo = find_bbo_1s(row['TradeTime'],row['primary_mic'])
    bbo_prices.append({'best_bid_1s': bbo[0], 'best_ask_1s': bbo[1]})

# Add BBO prices to the execution DataFrame
bbo_df = pd.DataFrame(bbo_prices)
execution_df = pd.concat([execution_df, bbo_df], axis=1)

    b. Mid-Price – bbo data in marketdata.parquet
        i. Find the Mid-Price at execution, 1s before the execution and 1s after the execution

In [29]:
# here i have made an assumption on what i thing is a mid_price.
execution_df['mid_price']=(execution_df['best_bid']+execution_df['best_ask'])/2
execution_df['mid_price_min_1s']=(execution_df['best_bid_min_1s']+execution_df['best_ask_min_1s'])/2
execution_df['mid_price_1s']=(execution_df['best_bid_1s']+execution_df['best_ask_1s'])/2

 c. Calculate Slippage [‘slippage’] at execution price

In [30]:
execution_df['slippage']=np.where(execution_df['Side']==2, execution_df['Price']-execution_df['best_bid']/execution_df['best_ask']-execution_df['best_bid'],
                                  execution_df['best_ask']-execution_df['Price']/execution_df['best_ask']-execution_df['best_bid']
                                  )

In [31]:
execution_df.to_excel('./processed/berenberg.xlsx')
execution_df

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,Trade_Dt,Side,...,best_bid,best_ask,best_bid_min_1s,best_ask_min_1s,best_bid_1s,best_ask_1s,mid_price,mid_price_min_1s,mid_price_1s,slippage
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39,91.80,100,CONTINUOUS_TRADING,11,2022-09-02,1,...,91.75,91.85,40.45,40.47,40.450,40.470,91.800,40.460,40.4600,-0.899456
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05,91.90,101,CONTINUOUS_TRADING,16,2022-09-02,1,...,91.85,91.95,40.52,40.54,47.675,47.690,91.900,40.530,47.6825,-0.899456
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55,91.85,102,CONTINUOUS_TRADING,22,2022-09-02,1,...,91.80,91.95,40.44,40.47,40.440,40.470,91.875,40.455,40.4550,-0.848912
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54,91.95,103,CONTINUOUS_TRADING,17,2022-09-02,1,...,91.90,92.00,40.49,40.52,47.850,47.865,91.950,40.505,47.8575,-0.899457
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06,92.05,104,CONTINUOUS_TRADING,23,2022-09-02,1,...,92.00,92.15,30.66,30.70,47.875,47.890,92.075,30.680,47.8825,-0.848915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,DE0005552004,EUR,XETA,2022-09-02 15:29:56,36.66,4198,CONTINUOUS_TRADING,37,2022-09-02,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4099,DE0005552004,EUR,XETA,2022-09-02 15:29:58,36.65,4199,CONTINUOUS_TRADING,77,2022-09-02,1,...,NaN,NaN,NaN,NaN,22.780,22.880,NaN,NaN,22.8300,NaN
4100,DE0005552004,EUR,XETA,2022-09-02 15:35:26,36.60,4200,CONTINUOUS_TRADING,1279,2022-09-02,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4101,DE000A0LD6E6,EUR,XETA,2022-09-02 15:35:25,52.55,4201,CONTINUOUS_TRADING,-35,2022-09-02,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Total number of rows for the filtered Execution Data matches for Before and After

In [32]:
df_executions_filtered.shape[0] == execution_df.shape[0]

True

In [33]:
endtime=pd.Timestamp.now() 

In [34]:
total_time=endtime-startime

# 5. Please provide performance metrics on your program

In [39]:
# Primary Mic join slows the run, if we remove this join or include in index process is 90% faster and number of values populated will also 
# increase to 90% this can be changed by understanding the data more and its relation
total_time

Timedelta('0 days 00:19:57.595834')

# Assumtions/Caveats

In [36]:
#Some values in the output excel are not present as market data file has Primary Mic data missing
print(df_executions_filtered['primary_mic'].unique())
print(df_marketdata['primary_mic'].unique())

['XBRU' 'XETR' 'XSWX' 'XLON' 'XCSE']
['XBRU' 'XSWX' 'XETR']


In [37]:
#same for listing_id if we consider this for the joins
print(df_executions_filtered['id'].unique())
print(df_marketdata['listing_id'].unique())

[   328336    326330    329308    328259    335438    336193    323448
    326288    323508    329340    499639    326525    327489    327496
    323472    326295    324162    323411    326490    323496    325561
    325460    324084    343570    325453 286167907 286168571 378534881
 286168597    343630    324144    329527 377546522    331495    326218
    326232    326302    326455    326539    331530    325439    325502
    325509    325624    338253    329445    325516    325467    325544
    325432    325474    329480    327391    324168    324078    326176
    326400    323436    326309    326169    323502    326351    324072
    323478    336379    327538    331732]
[378534881    328259    324162    324168    324078    324072    324084
    328336    324144    323436    323502    323508    323478    323472
    323448    323496 286087398 286087258 286087896]


In [38]:
# Also some +1secs and -1secs values are missing as there is no timestamp value for +1 or -1 , this can be changed with further information on what to do in such cases like to take the closest value.